# The ProtGVP input is constructed using atomic coordinates, amino acid type, grouping result, and protbert characteristic file of peptide

In [2]:
import jsonlines
import numpy as np
import pandas as pd
import os

In [3]:
pep_file = './peptide6A_seq5_15_protbert.npy'#name features
seq_file =  './pocket6A_seq10_50.fasta'
coords_file = './gvp6A_primary.jsonl'
neg_file = './pocketonly_classed_five_neg_c6.npy'

seq_dict ={}#蛋白序列
with open(seq_file) as f:
    lines= f.readline()
    line_1 = lines.strip()
    lines= f.readline()
    line_2 = lines.strip()
    while lines:
        seq_dict[line_1[1:]]=line_2
        lines= f.readline()
        line_1 = lines.strip()
        lines= f.readline()
        line_2 = lines.strip()
        
pep_dict ={}#多肽protbert
pep_data = np.load(pep_file,allow_pickle=True)
for item in pep_data:
    pep_dict[item['name']] = item['features']
    
coords_dict = {}#坐标
with open(coords_file, "r+", encoding="utf8") as f:
    for item in jsonlines.Reader(f):
        coords_dict[item['name']] = item['coords']
        
neg_dict ={}#负例
neg_data = np.load(neg_file,allow_pickle=True)
for class_i in neg_data:
    neg_dict.update(class_i)      

In [7]:
#输入数据,输入自己模型的数据，保证长度为50和15以内,不过mask和特征都变成list了
item = {}
max_pro_length = 50
max_pep_length =15
#pep_path = '../new_data/pep_protbert'
for key in neg_dict.keys():
    item = {}
    item['seq'] = seq_dict[key]
    pro_mask = np.zeros(max_pro_length).astype(bool) # mask
    pro_mask[:len(item['seq'])]=True # mask
    item['pro_mask'] = pro_mask.tolist()
    list_N = []
    list_CA = []
    list_C = []
    list_O = []
    for coor in coords_dict[key]:
        list_N.append(coor[0])
        list_CA.append(coor[1])
        list_C.append(coor[2])
        list_O.append(coor[3])
    ii ={}
    if len(list_C)!=len(item['seq']):
        print(key)
        print(len(list_C))
        print(len(item['seq']))
        continue
    ii['N'] = list_N 
    ii['CA'] = list_CA
    ii['C'] = list_C
    ii['O'] = list_O
    item['coords'] = ii
    item['name'] = key
    
    

    pep_data=pep_dict[key]
    pep_length = len(pep_data)
    # n*1024  (np.float32)会损失精度
    pep_codes = np.array(pep_data).reshape(-1, 1024)
    pep_code = np.pad(pep_codes,((0,max_pep_length-pep_length),(0,0)),'constant',constant_values=(0,0)).astype(np.float32)
    pep_mask=np.zeros(max_pep_length).astype(bool) # mask
    pep_mask[:pep_length]=True # mask
    
    item['pep_feature'] = pep_code.tolist()
    item['pep_mask'] = pep_mask.tolist()
    item['label'] = 1
    
    #写入
    #print(item)
    with jsonlines.open('gvp_data6A.jsonl', mode='a') as writer:
        writer.write(item)
        
    #负例
    pep_data=pep_dict[neg_dict[key]]
    pep_length = len(pep_data)
    # n*1024  (np.float32)会损失精度
    pep_codes = np.array(pep_data).reshape(-1, 1024)
    pep_code = np.pad(pep_codes,((0,max_pep_length-pep_length),(0,0)),'constant',constant_values=(0,0)).astype(np.float32)
    pep_mask=np.zeros(max_pep_length).astype(bool) # mask
    pep_mask[:pep_length]=True # mask
    
    item['pep_feature'] = pep_code.tolist()
    item['pep_mask'] = pep_mask.tolist()
    item['label'] = 0
    #item['pep_name'] = str(pep_name[:-4]) + 'neg.csv'
    with jsonlines.open('gvp_data6A.jsonl', mode='a') as writer:
        writer.write(item)

5w93_D_A.txt
24
25
4p4w_A.txt
18
19
5lgs_H_C.txt
26
27
5dx1_I_C.txt
29
30
5dx1_G_A.txt
28
29
4qj6_F_B.txt
42
43
4obg_F_B.txt
42
43
4obf_F_B.txt
39
40
5k99_C_A.txt
14
15
1nam_P.txt
38
39
5how_A.txt
10
11
1qow_A.txt
16
17
6muo_L.txt
23
24
1q4q_T_B.txt
26
27


# Document training, validation, and testing

In [11]:
#分组用，2，3分别的测试和验证，其他为训练
###########



#只用口袋分组



############
name_dict={}
name_dict['train'] = []
name_dict['validation'] = []
name_dict['test']=[]
neg_file = './pocketonly_classed_five_c6.npy'
neg_data = np.load(neg_file,allow_pickle=True)
for i in range(5):
    if i==3:
        name_dict['test'] =  neg_data[i]
    elif i==2:
        name_dict['validation'] = neg_data[i]
    else:
        name_dict['train'].extend(neg_data[i])
with jsonlines.open('pocketonly_split_23.jsonl', mode='w') as writer:
    writer.write(name_dict)  